## Modelos de Predicción para Credifiel Equipo Beta

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [23]:
# Cargar archivo
archivo = "Master.xlsx"

# Cargar hojas de detalle y unir
hojas_detalle = [
    "ListaCobroDetalle2022",
    "ListaCobroDetalle2023",
    "ListaCobroDetalle2024",
    "ListaCobroDetalle2025"
]

df_detalle = pd.concat([pd.read_excel(archivo, sheet_name=hoja) for hoja in hojas_detalle], ignore_index=True)

# Cargar catálogos
cat_banco = pd.read_excel(archivo, sheet_name="CatBanco")
cat_emisora = pd.read_excel(archivo, sheet_name="CatEmisora")
cat_respuesta = pd.read_excel(archivo, sheet_name="CatRespuestaBancos")
df_cobros = pd.read_excel(archivo, sheet_name="ListaCobro")


try:
     #encoding='utf-8'
    df_pagares = pd.read_csv("Pagares.csv", sep=',', encoding='utf-8')
except:
    try:
        #sep=';'
        df_pagares = pd.read_csv("Pagares.csv", sep=';', encoding='utf-8')
    except:
        try:
            #encoding='latin1'
            df_pagares = pd.read_csv("Pagares.csv", encoding='latin1')
        except:
            # Excel
            df_pagares = pd.read_excel("Pagares.csv")

# Mostrar las primeras
print(df_pagares.head())

   Credito_pag  CapInicial    Pagare  FechaApertura
0       694285     18370.0  51612.00          45334
1       686968     24747.0  50375.04          45260
2       747675      2400.0   6743.04          45723
3       736749     20000.0  56191.68          45625
4       650810     29946.0  60958.08          44984


In [84]:
#Conivertimos a texto
df_cobros['idListaCobro'] = df_cobros['idListaCobro'].astype(str).str.strip()
df_detalle['idListaCobro'] = df_detalle['idListaCobro'].astype(str).str.strip()

#Quita cosas anteriores 
df_detalle = df_detalle.drop(columns=[col for col in df_detalle.columns if 'fechaCreacionLista' in col], errors='ignore')

#Merge con df_cobros para agregar la fecha del pago
df_detalle = df_detalle.merge(
    df_cobros[['idListaCobro', 'fechaCreacionLista']],
    on='idListaCobro',
    how='left'
)

#Formato datetime.
df_detalle['fechaCreacionLista'] = pd.to_datetime(df_detalle['fechaCreacionLista'], errors='coerce', dayfirst=True)

#Numeración y Orden 
df_detalle = df_detalle.sort_values(by=['idCredito', 'fechaCreacionLista'])
df_detalle['numIntento'] = df_detalle.groupby('idCredito').cumcount() + 1

#Variables tiempo salen de la fecha de transacción
df_detalle['horaIntento'] = df_detalle['fechaCreacionLista'].dt.hour
df_detalle['diaSemana'] = df_detalle['fechaCreacionLista'].dt.dayofweek
df_detalle['diaMes'] = df_detalle['fechaCreacionLista'].dt.day

df_detalle



,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,banco_monto_medio_y,banco_monto_std_y,banco_tasa_exito_y,tipoCanal,idEmisora,TipoEnvio,CapInicial_pagare,Pagare_pagare,FechaApertura_pagare,fechaCreacionLista
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,9.789460,122.651711,0.012942,5,9,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:36:00
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,20.270358,183.148854,0.026090,5,9,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:36:00
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,20.270358,183.148854,0.026090,2,62,PARCIAL,NaN,NaN,NaN,2023-01-30 07:34:00
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,20.270358,183.148854,0.026090,2,62,PARCIAL,NaN,NaN,NaN,2023-03-27 08:18:00
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,87.333148,324.055181,0.128769,5,7,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,9.789460,122.651711,0.012942,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-16 12:04:00
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,9.789460,122.651711,0.012942,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-17 06:12:00
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,9.789460,122.651711,0.012942,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-17 10:04:00
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,9.789460,122.651711,0.012942,7,2,CUENTA,NaN,NaN,NaN,2025-01-17 12:54:00


In [138]:


# Elimina columnas  df_detalle que contengan el credito pag
df_detalle = df_detalle.drop(columns=[col for col in df_detalle.columns if 'Credito_pag' in col], errors='ignore')

# Ids mismo formato
df_detalle['idCredito'] = df_detalle['idCredito'].astype(str).str.strip()
df_pagares['Credito_pag'] = df_pagares['Credito_pag'].astype(str).str.strip()

# No se crea conflicto por columnas duplicadas
df_detalle = df_detalle.merge(
    df_pagares[['Credito_pag', 'CapInicial', 'Pagare', 'FechaApertura']].rename(columns={
        'CapInicial': 'CapInicial_pagare',
        'Pagare': 'Pagare_pagare', 
        'FechaApertura': 'FechaApertura_pagare'
    }),
    left_on='idCredito',
    right_on='Credito_pag',
    how='left'
)

# Quita columna credito pag
df_detalle = df_detalle.drop(columns=['Credito_pag'], errors='ignore')

: 

In [86]:
# 5. Crear variables financieras
df_detalle['ratio_cobro'] = df_detalle['montoCobrado'] / df_detalle['montoCobrar']
df_detalle['ratio_pagare'] = df_detalle['Pagare'] / df_detalle['CapInicial']

# 6. Verificar las nuevas variables
print("\nEstadísticas de los ratios:")
print(df_detalle[['ratio_cobro', 'ratio_pagare']].describe())

df_detalle


Estadísticas de los ratios:
        ratio_cobro  ratio_pagare
count  3.506737e+06  1.348359e+06
mean   3.482267e-02  2.345770e+00
std    1.832758e-01  3.705036e-01
min    0.000000e+00  1.474772e+00
25%    0.000000e+00  2.035602e+00
50%    0.000000e+00  2.035617e+00
75%    0.000000e+00  2.809586e+00
max    2.500061e+00  2.809685e+00


,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,tipoCanal,idEmisora,TipoEnvio,CapInicial_pagare_x,Pagare_pagare_x,FechaApertura_pagare_x,fechaCreacionLista,CapInicial_pagare_y,Pagare_pagare_y,FechaApertura_pagare_y
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,5,9,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:36:00,NaN,NaN,NaN
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,5,9,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:36:00,NaN,NaN,NaN
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,2,62,PARCIAL,NaN,NaN,NaN,2023-01-30 07:34:00,NaN,NaN,NaN
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,2,62,PARCIAL,NaN,NaN,NaN,2023-03-27 08:18:00,NaN,NaN,NaN
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,5,7,TRADICIONAL,NaN,NaN,NaN,2022-10-25 05:28:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-16 12:04:00,NaN,NaN,NaN
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-17 06:12:00,NaN,NaN,NaN
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,5,5,TRADICIONAL,NaN,NaN,NaN,2025-01-17 10:04:00,NaN,NaN,NaN
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,7,2,CUENTA,NaN,NaN,NaN,2025-01-17 12:54:00,NaN,NaN,NaN


In [87]:

df_detalle['es_quincena'] = (df_detalle['diaMes'].isin([15, 30, 31])).astype(int)
df_detalle['es_inicio_mes'] = (df_detalle['diaMes'] <= 5).astype(int)
df_detalle['es_fin_mes'] = (df_detalle['diaMes'] >= 25).astype(int)

print("\nVariables temporales creadas:")
print(df_detalle[['es_quincena', 'es_inicio_mes', 'es_fin_mes']].sum())


Variables temporales creadas:
es_quincena       406914
es_inicio_mes     343035
es_fin_mes       1450574
dtype: int64


In [ ]:

df_lista_emisora = pd.read_csv("ListaCobroEmisora.csv")  


df_lista_emisora['idListaCobro'] = df_lista_emisora['idListaCobro'].astype(str).str.strip()
df_lista_emisora['idEmisora'] = df_lista_emisora['idEmisora'].astype(str).str.strip()
df_detalle['idListaCobro'] = df_detalle['idListaCobro'].astype(str).str.strip()
cat_emisora['idEmisora'] = cat_emisora['idEmisora'].astype(str).str.strip()
cat_emisora['TipoEnvio'] = cat_emisora['TipoEnvio'].astype(str).str.upper().str.strip()


df_detalle = df_detalle.drop(columns=[col for col in df_detalle.columns if 'idEmisora' in col or 'TipoEnvio' in col], errors='ignore')


df_detalle = df_detalle.merge(
    df_lista_emisora[['idListaCobro', 'idEmisora']],
    on='idListaCobro',
    how='left'
)


df_detalle = df_detalle.merge(
    cat_emisora[['idEmisora', 'TipoEnvio']],
    on='idEmisora',
    how='left'
)


mapa_tipoenvio = {
    'INTERBANCARIO': 1,
    'PARCIAL': 2,
    'REINTENTO': 3,
    'EXCEPCION': 4,
    'TRADICIONAL': 5,
    'TARJETA': 6,
    'CUENTA': 7,
    'MATUTINO': 8,
    'EN LINEA': 9
}
df_detalle['tipoCanal'] = df_detalle['TipoEnvio'].map(mapa_tipoenvio).fillna(0).astype(int)

df_detalle

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,tipoCanal,CapInicial_pagare_x,Pagare_pagare_x,FechaApertura_pagare_x,fechaCreacionLista,CapInicial_pagare_y,Pagare_pagare_y,FechaApertura_pagare_y,idEmisora,TipoEnvio
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,5,NaN,NaN,NaN,2022-10-25 05:36:00,NaN,NaN,NaN,9,TRADICIONAL
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,5,NaN,NaN,NaN,2022-10-25 05:36:00,NaN,NaN,NaN,9,TRADICIONAL
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,2,NaN,NaN,NaN,2023-01-30 07:34:00,NaN,NaN,NaN,62,PARCIAL
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,2,NaN,NaN,NaN,2023-03-27 08:18:00,NaN,NaN,NaN,62,PARCIAL
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,5,NaN,NaN,NaN,2022-10-25 05:28:00,NaN,NaN,NaN,7,TRADICIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,5,NaN,NaN,NaN,2025-01-16 12:04:00,NaN,NaN,NaN,5,TRADICIONAL
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,5,NaN,NaN,NaN,2025-01-17 06:12:00,NaN,NaN,NaN,5,TRADICIONAL
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,5,NaN,NaN,NaN,2025-01-17 10:04:00,NaN,NaN,NaN,5,TRADICIONAL
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,7,NaN,NaN,NaN,2025-01-17 12:54:00,NaN,NaN,NaN,2,CUENTA


In [93]:

df_detalle['exitoRedefinido'] = (df_detalle['idRespuestaBanco'] == 0).astype(int)


df_detalle['ratio_cobro'] = df_detalle['montoCobrado'] / df_detalle['montoCobrar']
df_detalle['ratio_pagare'] = df_detalle['Pagare_pagare_y'] / df_detalle['CapInicial_pagare_y']

df_detalle['es_quincena'] = df_detalle['diaMes'].isin([15, 30, 31]).astype(int)
df_detalle['es_inicio_mes'] = (df_detalle['diaMes'] <= 5).astype(int)
df_detalle['es_fin_mes'] = (df_detalle['diaMes'] >= 25).astype(int)


stats_banco = df_detalle.groupby('idBanco').agg({
    'montoCobrado': ['mean', 'std'],
    'exitoRedefinido': 'mean'
}).reset_index()
stats_banco.columns = ['idBanco', 'banco_monto_medio', 'banco_monto_std', 'banco_tasa_exito']
df_detalle = df_detalle.merge(stats_banco, on='idBanco', how='left')


stats_tipo = df_detalle.groupby('TipoEnvio').agg({
    'exitoRedefinido': 'mean'
}).reset_index()
stats_tipo.columns = ['TipoEnvio', 'tipo_tasa_exito']
df_detalle = df_detalle.merge(stats_tipo, on='TipoEnvio', how='left')


stats_emisora = df_detalle.groupby('idEmisora').agg({
    'exitoRedefinido': 'mean'
}).reset_index()
stats_emisora.columns = ['idEmisora', 'emisora_tasa_exito']
df_detalle = df_detalle.merge(stats_emisora, on='idEmisora', how='left')

In [ ]:

df_detalle['TipoEnvio'] = df_detalle['TipoEnvio'].astype(str).str.upper().str.strip()

df_detalle['TipoEnvio'] = df_detalle['TipoEnvio'].apply(lambda x: 'TRADICIONAL' if x == 'TRADICIONAL' else 'NO_TRADICIONAL')


print(df_detalle['TipoEnvio'].value_counts())

TipoEnvio
TRADICIONAL       1892370
NO_TRADICIONAL    1614367
Name: count, dtype: int64


In [102]:


df_detalle.drop(columns=['TipoEnvio'], inplace=True)
df_detalle


,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,CapInicial_pagare_y,Pagare_pagare_y,FechaApertura_pagare_y,idEmisora,banco_monto_medio,banco_monto_std,banco_tasa_exito,tipo_tasa_exito,emisora_tasa_exito,es_tradicional
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,NaN,NaN,NaN,9,9.789460,122.651711,0.012942,0.036226,0.194770,1
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,NaN,NaN,NaN,9,20.270358,183.148854,0.026090,0.036226,0.194770,1
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,NaN,NaN,NaN,62,20.270358,183.148854,0.026090,0.048186,0.248204,0
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,NaN,NaN,NaN,62,20.270358,183.148854,0.026090,0.048186,0.248204,0
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,NaN,NaN,NaN,7,87.333148,324.055181,0.128769,0.036226,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,NaN,NaN,NaN,5,9.789460,122.651711,0.012942,0.036226,0.019062,1
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,NaN,NaN,NaN,5,9.789460,122.651711,0.012942,0.036226,0.019062,1
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,NaN,NaN,NaN,5,9.789460,122.651711,0.012942,0.036226,0.019062,1
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,NaN,NaN,NaN,2,9.789460,122.651711,0.012942,0.022451,0.020447,0


In [105]:
df_detalle['horaDomiciliacion'] = df_detalle['fechaCreacionLista'].dt.hour

def clasificar_bloque(hora):
    if pd.isna(hora):
        return 'DESCONOCIDO'
    elif hora < 8:
        return 'MADRUGADA'
    elif hora < 12:
        return 'MAÑANA'
    elif hora < 16:
        return 'MEDIODIA'
    elif hora < 20:
        return 'TARDE'
    else:
        return 'NOCHE'

df_detalle['bloqueDomiciliacion'] = df_detalle['horaDomiciliacion'].apply(clasificar_bloque)


mapa_bloque = {
    'MADRUGADA': 1,
    'MAÑANA': 2,
    'MEDIODIA': 3,
    'TARDE': 4,
    'NOCHE': 5,
    'DESCONOCIDO': 0
}
df_detalle['bloqueDomiciliacion_num'] = df_detalle['bloqueDomiciliacion'].map(mapa_bloque)

In [111]:
df_detalle

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,banco_monto_std,banco_tasa_exito,tipo_tasa_exito,emisora_tasa_exito,es_tradicional,horaDomiciliacion,bloqueDomiciliacion,bloqueDomiciliacion_num,TipoEnvio_x,TipoEnvio_y
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,122.651711,0.012942,0.036226,0.194770,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,183.148854,0.026090,0.036226,0.194770,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,183.148854,0.026090,0.048186,0.248204,0,7,MADRUGADA,1,PARCIAL,PARCIAL
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,183.148854,0.026090,0.048186,0.248204,0,8,MAÑANA,2,PARCIAL,PARCIAL
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,324.055181,0.128769,0.036226,0.000000,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,122.651711,0.012942,0.036226,0.019062,1,12,MEDIODIA,3,TRADICIONAL,TRADICIONAL
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,122.651711,0.012942,0.036226,0.019062,1,6,MADRUGADA,1,TRADICIONAL,TRADICIONAL
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,122.651711,0.012942,0.036226,0.019062,1,10,MAÑANA,2,TRADICIONAL,TRADICIONAL
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,122.651711,0.012942,0.022451,0.020447,0,12,MEDIODIA,3,CUENTA,CUENTA


In [ ]:

df_detalle['idEmisora'] = df_detalle['idEmisora'].astype(str).str.strip()
cat_emisora['idEmisora'] = cat_emisora['idEmisora'].astype(str).str.strip()
cat_emisora['TipoEnvio'] = cat_emisora['TipoEnvio'].str.upper().str.strip()


df_detalle = df_detalle.merge(
    cat_emisora[['idEmisora', 'TipoEnvio']],
    on='idEmisora',
    how='left'
)


mapa_tipoenvio = {
    'INTERBANCARIO': 1,
    'PARCIAL': 2,
    'REINTENTO': 3,
    'EXCEPCION': 4,
    'TRADICIONAL': 5,
    'TARJETA': 6,
    'CUENTA': 7,
    'MATUTINO': 8,
    'EN LINEA': 9
}

df_detalle['idTipoEnvio'] = df_detalle['TipoEnvio'].map(mapa_tipoenvio).fillna(0).astype(int)

df_detalle

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,es_tradicional,horaDomiciliacion,bloqueDomiciliacion,bloqueDomiciliacion_num,TipoEnvio_x_x,TipoEnvio_y,TipoEnvio_x_y,TipoEnvio_x,idTipoEnvio,TipoEnvio
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,0,7,MADRUGADA,1,PARCIAL,PARCIAL,PARCIAL,PARCIAL,2,PARCIAL
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,0,8,MAÑANA,2,PARCIAL,PARCIAL,PARCIAL,PARCIAL,2,PARCIAL
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,1,12,MEDIODIA,3,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,1,6,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,1,10,MAÑANA,2,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,0,12,MEDIODIA,3,CUENTA,CUENTA,CUENTA,CUENTA,7,CUENTA


In [124]:
df_detalle

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,numIntento,...,es_tradicional,horaDomiciliacion,bloqueDomiciliacion,bloqueDomiciliacion_num,TipoEnvio_x_x,TipoEnvio_y,TipoEnvio_x_y,TipoEnvio_x,idTipoEnvio,TipoEnvio
0,102697,10073,29066732,2,1337.56,1337.56,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
1,102697,101713,29089815,12,2261.68,2261.68,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
2,110918,101713,30408869,12,2261.68,2261.68,0.0,NaN,13,2,...,0,7,MADRUGADA,1,PARCIAL,PARCIAL,PARCIAL,PARCIAL,2,PARCIAL
3,115660,101713,31028638,12,746.35,746.35,0.0,NaN,13,3,...,0,8,MAÑANA,2,PARCIAL,PARCIAL,PARCIAL,PARCIAL,2,PARCIAL
4,102693,101797,29058467,14,1034.49,1034.49,0.0,NaN,NaN,1,...,1,5,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506732,156711,9872,41688488,2,484.60,484.60,0.0,NaN,4.0,495,...,1,12,MEDIODIA,3,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506733,156727,9872,41691792,2,484.60,484.60,0.0,NaN,4.0,496,...,1,6,MADRUGADA,1,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506734,156736,9872,41694545,2,484.60,484.60,0.0,NaN,4.0,497,...,1,10,MAÑANA,2,TRADICIONAL,TRADICIONAL,TRADICIONAL,TRADICIONAL,5,TRADICIONAL
3506735,156743,9872,41700604,2,484.60,484.60,0.0,NaN,4.0,498,...,0,12,MEDIODIA,3,CUENTA,CUENTA,CUENTA,CUENTA,7,CUENTA


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report


# Solo los segundos intentos
df_modelo = df_detalle[df_detalle['numIntento'] == 2].copy()

#  Definir variables para el modelo
columnas_modelo = [
    'es_tradicional', 'montoExigible', 'montoCobrar', 'montoCobrado',
    'horaIntento', 'diaSemana', 'diaMes', 'idBanco',
    'ratio_cobro', 'ratio_pagare',
    'es_quincena', 'es_inicio_mes', 'es_fin_mes',
    'banco_monto_medio', 'banco_monto_std', 'banco_tasa_exito',
    'tipo_tasa_exito', 'emisora_tasa_exito',
    'CapInicial_pagare_y', 'Pagare_pagare_y', 'idTipoEnvio'
]

# Separar datos
X = df_modelo[columnas_modelo].dropna()
y = df_modelo.loc[X.index, 'exitoRedefinido']

#Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Entrenar el modelo
modelo_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight='balanced'
)
modelo_rf.fit(X_train, y_train)

# Evaluar desempeño
y_pred = modelo_rf.predict(X_test)

print("\n Resultados del Modelo:")
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))

#Importancia de características
importancia = pd.DataFrame({
    'caracteristica': X.columns,
    'importancia': modelo_rf.feature_importances_
}).sort_values('importancia', ascending=False)

#Visualización
plt.figure(figsize=(12, 6))
sns.barplot(data=importancia.head(10), x='importancia', y='caracteristica')
plt.title('10 Variables Más Importantes')
plt.tight_layout()
plt.show()

NameError: name 'df_detalle' is not defined

In [ ]:
# Escenario 3A: Bajo costo y alta probabilidad de éxito
condiciones_3a = (
    (df_modelo['costo_envio'] <= 2.00) &
    (df_modelo['predicho'] == 1) &
    (df_modelo['ratio_cobro'] >= 0.85) &
    (df_modelo['montoCobrar'] <= 2500) &
    (df_modelo['horaIntento'].between(8, 14)) &
    (df_modelo['diaMes'].isin([1, 2, 3, 4, 5, 25, 26, 27, 28, 29, 30, 31])) &
    (df_modelo['idTipoEnvio'].isin([5, 6, 7]))  # TRADICIONAL, CUENTA, TARJETA
)

escenario_3a = df_modelo[condiciones_3a]

# Escenario 3B: Máxima cobertura sin importar costo
escenario_3b = df_modelo[df_modelo['predicho'] == 1]

# Resultados
print("Escenario 3A — Bajo Costo")
print("Registros:", len(escenario_3a))
print("Tasa de éxito esperada:", escenario_3a['exitoRedefinido'].mean())

print("\nEscenario 3B — Máxima Cobranza")
print("Registros:", len(escenario_3b))
print("Tasa de éxito esperada:", escenario_3b['exitoRedefinido'].mean())

📌 Escenario 3A — Bajo Costo
Registros: 1049
Tasa de éxito esperada: 0.9914204003813155

📌 Escenario 3B — Máxima Cobranza
Registros: 8051
Tasa de éxito esperada: 0.820394981989815
